# Deploying Merlin Query Tower with Vertex AI

* Create custom prediction routine (CPR)
* Upload query model to Vertex AI Model Registry
* Test registered models predictions

In [95]:
from google.cloud import aiplatform as vertex_ai
import os

PROJECT_ID = 'hybrid-vertex'             
LOCATION = 'us-central1' 
BUCKET = 'jt-merlin-scaling'
BUCKET_URI = 'gs://jt-merlin-scaling'

vertex_ai.init(project=PROJECT_ID, location=LOCATION)
# !gcloud config set project {PROJECT_ID}

In [96]:
# !gcloud auth application-default login

## Build serving container

In [97]:
REPO_DOCKER_PATH_PREFIX = 'src'
SERVING_SUB_DIR = 'serving'
SERVING_APPLICATION_DIR = 'app'
SERVING_DOCKERNAME = 'merlin-retriever'

In [98]:
# Make the training subfolder
! rm -rf {REPO_DOCKER_PATH_PREFIX}/{SERVING_SUB_DIR}
! mkdir {REPO_DOCKER_PATH_PREFIX}/{SERVING_SUB_DIR}
! mkdir {REPO_DOCKER_PATH_PREFIX}/{SERVING_SUB_DIR}/{SERVING_APPLICATION_DIR}
! touch {REPO_DOCKER_PATH_PREFIX}/{SERVING_SUB_DIR}/{SERVING_APPLICATION_DIR}/__init__.py

nvtabular==1.3.3 (for prediction only per ronnay AK)

In [99]:
%%writefile {REPO_DOCKER_PATH_PREFIX}/{SERVING_SUB_DIR}/requirements.txt
uvicorn[standard]==0.15.0
gunicorn==20.1.0
fastapi==0.68.1
uvloop==0.15.2
fastapi-utils
google-cloud-aiplatform
git+https://github.com/NVIDIA-Merlin/models.git
nvtabular==1.3.3
gcsfs
google-cloud-storage

Writing src/serving/requirements.txt


In [100]:
%%writefile {REPO_DOCKER_PATH_PREFIX}/{SERVING_SUB_DIR}/{SERVING_APPLICATION_DIR}/predictor.py
import nvtabular as nvt
import pandas as pd
import os
import json
import merlin.models.tf as mm
from nvtabular.loader.tf_utils import configure_tensorflow
configure_tensorflow()
import tensorflow as tf
import time
import logging


# These are helper functions that ensure the dictionary input is in a certain order and types are preserved
# this is to get scalar values to appear first in the dict to not confuse pandas with lists https://github.com/pandas-dev/pandas/issues/46092
reordered_keys = [
    'collaborative', 
    'album_name_pl', 
    'artist_genres_pl', 
    'artist_name_pl', 
    'artist_pop_can', 
    'description_pl', 
    'duration_ms_songs_pl', 
    'n_songs_pl', 'name', 
    'num_albums_pl', 
    'num_artists_pl', 
    'track_name_pl', 
    'track_pop_pl', 
    'duration_ms_seed_pl', 
    'pid', 
    'track_uri_pl'
]

float_num_fix = ['n_songs_pl','num_albums_pl','num_artists_pl','duration_ms_seed_pl']
float_list_fix = ['track_pop_pl', 'duration_ms_songs_pl']
    
def fix_list_num_dtypes(num_list):
    "this fixes lists of ints to list of floats converted in json input"
    return [float(x) for x in num_list]

def fix_num_dtypes(num):
    "this fixes ints and casts to floats"
    return float(num)

def fix_types(k, v):
    if k in float_num_fix:
        return fix_num_dtypes(v)
    if k in float_list_fix:
        return fix_list_num_dtypes(v)
    else:
        return v

def create_pandas_instance(inputs):
    """
    Helper function to reorder the input to have a sclar first for pandas
    And fix the types converted when data is imported by fastAPI
    """
    if type(inputs) == list:
        header = inputs[0]
        reordered_header_dict = {k: fix_types(k,header[k]) for k in reordered_keys}
        pandas_instance = pd.DataFrame.from_dict(reordered_header_dict, orient='index').T
        if len(inputs) > 1:
            for ti in inputs[1:]:
                reordered_dict = {k: fix_types(k,ti[k]) for k in reordered_keys}
                pandas_instance = pandas_instance.append(pd.DataFrame.from_dict(reordered_dict, orient='index').T)
    else:
        reordered_dict = {k: fix_types(k,inputs[k]) for k in reordered_keys}
        pandas_instance = pd.DataFrame.from_dict(reordered_dict, orient='index').T
    return pandas_instance

class Predictor():
    """Interface of the Predictor class for Custom Prediction Routines.
    The Predictor is responsible for the ML logic for processing a prediction request.
    Specifically, the Predictor must define:
    (1) How to load all model artifacts used during prediction into memory.
    (2) The logic that should be executed at predict time.
    When using the default PredictionHandler, the Predictor will be invoked as follows:
      predictor.postprocess(predictor.predict(predictor.preprocess(prediction_input)))
    """
    def __init__(self):
        return
    
    def load(self, artifacts_uri):
        """Loads the model artifact.
        Args:
            artifacts_uri (str):
                Required. The value of the environment variable AIP_STORAGE_URI.
        """
        logging.info("loading model and workflow")
        start = time.process_time()
        
        test_bucket = 'gs://jt-merlin-scaling'
        self.model = tf.keras.models.load_model(os.path.join(artifacts_uri, "query-tower"))
        # self.workflow = nvt.Workflow.load(os.path.join(artifacts_uri, "workflow/2t-spotify-workflow")) # TODO: parameterize
        self.workflow = nvt.Workflow.load(os.path.join(test_bucket, "nvt-last5-v1full/nvt-analyzed"))
        # self.workflow = nvt.Workflow.load('gs://jt-merlin-scaling/nvt-last5-v1full/nvt-analyzed') # TODO: parametrize
        self.workflow = self.workflow.remove_inputs(
            [
                'track_pop_can', 
                'track_uri_can', 
                'duration_ms_can', 
                'track_name_can', 
                'artist_name_can',
                'album_name_can',
                'album_uri_can',
                'artist_followers_can', 
                'artist_genres_can',
                'artist_name_can', 
                'artist_pop_can',
                'artist_pop_pl',
                'artist_uri_can', 
                'artists_followers_pl'
            ]
        )
        # self.model = tf.keras.models.load_model(os.path.join(artifacts_uri, "query_model_merlin" ))
        # self.workflow = nvt.Workflow.load(os.path.join(artifacts_uri, "workflow/2t-spotify-workflow"))
        # self.workflow = self.workflow.remove_inputs(['track_pop_can', 'track_uri_can', 'duration_ms_can', 
        #                               'track_name_can', 'artist_name_can','album_name_can',
        #                               'album_uri_can','artist_followers_can', 'artist_genres_can',
        #                               'artist_name_can', 'artist_pop_can','artist_pop_pl','artist_uri_can', 
        #                               'artists_followers_pl']) 
        self.loader = None # will load this after first load
        self.n_rows = 0
        logging.info(f"loading took {time.process_time() - start} seconds")
        
        return self
        
    def predict(self, prediction_input):
        """Preprocesses the prediction input before doing the prediction.
        Args:
            prediction_input (Any):
                Required. The prediction input that needs to be preprocessed.
        Returns:
            The preprocessed prediction input.
        """
        # handle different input types, can take a dict or list of dicts
        self.n_rows = len(prediction_input)
        start = time.process_time()
        pandas_instance = create_pandas_instance(prediction_input[0])
        logging.info(f"Pandas conversion took {time.process_time() - start} seconds")
        start = time.process_time()
        transformed_inputs = nvt.Dataset(pandas_instance)
        logging.info(f"NVT data loading took {time.process_time() - start} seconds")
        start = time.process_time()
        transformed_instance = self.workflow.transform(transformed_inputs)
        logging.info(f"Workflow transformation took {time.process_time() - start} seconds")
        # return transformed_instance

    # def predict(self, instances):
        """Performs prediction.
        Args:
            instances (Any):
                Required. The instance(s) used for performing prediction.
        Returns:
            Prediction results.
        """  
        # if self.loader is None:
        start = time.process_time()
        # if self.loader is None:
        #     self.loader = mm.Loader(transformed_instance, batch_size=1, shuffle=False)
        #     logging.info(f"Dataloader creation took {time.process_time() - start} seconds")
        # else:
        #     self.loader.data = transformed_instance #this is faster we don't want tokeep loading dataloder
        #     logging.info(f"Dataloader creation took {time.process_time() - start} seconds")
        start = time.process_time()
        batch = mm.sample_batch(transformed_instance, batch_size=1, include_targets=False, shuffle=False)
        logging.info(f"TF Dataloader took {time.process_time() - start} seconds")
        start = time.process_time()
        output = self.model(batch)
        logging.info(f"Prediction took {time.process_time() - start} seconds")
        return output

Writing src/serving/app/predictor.py


In [101]:
%%writefile {REPO_DOCKER_PATH_PREFIX}/{SERVING_SUB_DIR}/{SERVING_APPLICATION_DIR}/main.py
from fastapi import FastAPI, Request

import json
import numpy as np
import os
import logging
from fastapi_utils.timing import add_timing_middleware, record_timing

from google.cloud import storage
from .predictor import Predictor

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

predictor_instance = Predictor()
loaded_predictor = predictor_instance.load(artifacts_uri = os.environ['AIP_STORAGE_URI'])

app = FastAPI()
add_timing_middleware(app, record=logger.info, prefix="app", exclude="untimed")

@app.get(os.environ['AIP_HEALTH_ROUTE'], status_code=200)
def health():
    return {}


@app.post(os.environ['AIP_PREDICT_ROUTE'])
async def predict(request: Request):
    body = await request.json()
    instances = body["instances"]
    outputs = loaded_predictor.predict(instances)
    # outputs = loaded_predictor.predict(preprocessed_inputs)

    return {"predictions": outputs.numpy().tolist()}

Writing src/serving/app/main.py


In [102]:
%%writefile {REPO_DOCKER_PATH_PREFIX}/{SERVING_SUB_DIR}/{SERVING_APPLICATION_DIR}/prestart.sh
#!/bin/bash
export PORT=$AIP_HTTP_PORT

Writing src/serving/app/prestart.sh


In [103]:
%%writefile {REPO_DOCKER_PATH_PREFIX}/{SERVING_SUB_DIR}/instances.json
{"instances": {"collaborative": "false", "album_name_pl": ["There's Really A Wolf", "Late Nights: The Album", "American Teen", "Crazy In Love", "Pony"], "album_uri_can": "spotify:album:5l83t3mbVgCrIe1VU9uJZR", "artist_followers_can": 4339757.0, "artist_genres_can": "'hawaiian hip hop', 'rap'", "artist_genres_pl": ["'hawaiian hip hop', 'rap'", "'chicago rap', 'dance pop', 'pop', 'pop rap', 'r&b', 'southern hip hop', 'trap', 'urban contemporary'", "'pop', 'pop r&b'", "'dance pop', 'pop', 'r&b'", "'chill r&b', 'pop', 'pop r&b', 'r&b', 'urban contemporary'"], "artist_name_can": "Russ", "artist_name_pl": ["Russ", "Jeremih", "Khalid", "Beyonc\u00c3\u00a9", "William Singe"], "artist_pop_can": 82.0, "artist_pop_pl": [82.0, 80.0, 90.0, 87.0, 65.0], "artist_uri_can": "spotify:artist:1z7b1Pr1rSlvWRzsW3HOrS", "artists_followers_pl": [4339757.0, 5611842.0, 15046756.0, 30713126.0, 603837.0], "description_pl": "", "duration_ms_can": 237322.0, "duration_ms_songs_pl": [237506.0, 217200.0, 219080.0, 226400.0, 121739.0], "n_songs_pl": 8.0, "name": "Lit Tunes ", "num_albums_pl": 8.0, "num_artists_pl": 8.0, "track_name_can": "We Just Havent Met Yet", "track_name_pl": ["Losin Control", "Paradise", "Location", "Crazy In Love - Remix", "Pony"], "track_pop_can": 57.0, "track_pop_pl": [79.0, 58.0, 83.0, 71.0, 57.0], "duration_ms_seed_pl": 51023.1, "pid": 1, "track_uri_can": "spotify:track:0VzDv4wiuZsLsNOmfaUy2W", "track_uri_pl": ["spotify:track:4cxMGhkinTocPSVVKWIw0d", "spotify:track:1wNEBPo3nsbGCZRryI832I", "spotify:track:152lZdxL1OR0ZMW6KquMif", "spotify:track:2f4IuijXLxYOeBncS60GUD", "spotify:track:4Lj8paMFwyKTGfILLELVxt"]}}

Writing src/serving/instances.json


In [104]:
!pwd
!tree /home/jupyter/merlin-on-vertex

/home/jupyter/merlin-on-vertex
/home/jupyter/merlin-on-vertex
├── 01-data-preprocess-pipeline.ipynb
├── 02-merlin-vertex-training.ipynb
├── 03-query-model-inference.ipynb
├── 04-train-deploy-pipeline.ipynb
├── README.md
├── archive
│   └── archived-merlin-start.ipynb
├── custom_container_pipeline_spec.json
└── src
    ├── Dockerfile.merlin-retriever
    ├── Dockerfile.merlintf-22_09
    ├── Dockerfile.merlintf-22_09_v2
    ├── Dockerfile.triton-cpr
    ├── cloudbuild.yaml
    ├── pipes
    │   ├── config.py
    │   ├── pipe_components.py
    │   └── preproc_pipelines.py
    ├── preprocessor
    │   ├── __init__,py
    │   └── preprocess_task.py
    ├── serving
    │   ├── app
    │   │   ├── __init__.py
    │   │   ├── main.py
    │   │   ├── predictor.py
    │   │   └── prestart.sh
    │   ├── instances.json
    │   └── requirements.txt
    ├── train_pipes
    │   ├── build_custom_image.py
    │   ├── create_ann_index.py
    │   ├── create_ann_index_endpoint_vpc.py
    │   ├── create_

In [105]:
%%writefile {REPO_DOCKER_PATH_PREFIX}/Dockerfile.{SERVING_DOCKERNAME}

FROM nvcr.io/nvidia/merlin/merlin-tensorflow:22.09

WORKDIR / 

# COPY /src/serving/requirements.txt /requirements.txt
# COPY ./src/serving/requirements.txt /requirements.txt
COPY /serving/requirements.txt /requirements.txt

RUN pip install -r /requirements.txt

# COPY /src/serving/app /app
# COPY ./src/serving/app /app
COPY /serving/app /app

EXPOSE 80
    
CMD ["sh", "-c", "uvicorn app.main:app --host 0.0.0.0 --port $AIP_HTTP_PORT"]

Overwriting src/Dockerfile.merlin-retriever


## Copy serving app to GCS

**TODO**
> currently getting gsutil auth issues.. refactor this section with `gsutil -cp ...`

In [106]:
MODEL_DIR = 'gs://jt-merlin-scaling/test-2tower-merlin-tf-jtv1/run-v6-20221108-210323/model-dir'

In [107]:
!gsutil ls $MODEL_DIR

gs://jt-merlin-scaling/test-2tower-merlin-tf-jtv1/run-v6-20221108-210323/model-dir/
gs://jt-merlin-scaling/test-2tower-merlin-tf-jtv1/run-v6-20221108-210323/model-dir/Dockerfile.merlin-retriever
gs://jt-merlin-scaling/test-2tower-merlin-tf-jtv1/run-v6-20221108-210323/model-dir/candidate-embeddings/
gs://jt-merlin-scaling/test-2tower-merlin-tf-jtv1/run-v6-20221108-210323/model-dir/candidate-tower/
gs://jt-merlin-scaling/test-2tower-merlin-tf-jtv1/run-v6-20221108-210323/model-dir/query-tower/
gs://jt-merlin-scaling/test-2tower-merlin-tf-jtv1/run-v6-20221108-210323/model-dir/serving/


In [108]:
!gsutil cp ./src/Dockerfile.$SERVING_DOCKERNAME $MODEL_DIR/

Copying file://./src/Dockerfile.merlin-retriever [Content-Type=application/octet-stream]...
/ [1 files][  440.0 B/  440.0 B]                                                
Operation completed over 1 objects/440.0 B.                                      


In [109]:
!gsutil -m cp -r ./$REPO_DOCKER_PATH_PREFIX/$SERVING_SUB_DIR $MODEL_DIR/

Copying file://./src/serving/requirements.txt [Content-Type=text/plain]...
Copying file://./src/serving/app/__init__.py [Content-Type=text/x-python]...    
Copying file://./src/serving/instances.json [Content-Type=application/json]...
Copying file://./src/serving/app/predictor.py [Content-Type=text/x-python]...   
Copying file://./src/serving/app/main.py [Content-Type=text/x-python]...        
Copying file://./src/serving/app/prestart.sh [Content-Type=text/x-sh]...        
/ [6/6 files][  9.9 KiB/  9.9 KiB] 100% Done                                    
Operation completed over 6 objects/9.9 KiB.                                      


### delete these

In [110]:
# # !gcloud auth login

# from google.cloud import storage
# from google.cloud.storage.bucket import Bucket
# from google.cloud.storage.blob import Blob

# def _upload_blob_gcs(gcs_uri, source_file_name, destination_blob_name, project):
#     """Uploads a file to GCS bucket"""
#     client = storage.Client(project=project)
#     blob = Blob.from_string(os.path.join(gcs_uri, destination_blob_name))
#     blob.bucket._client = client
#     blob.upload_from_filename(source_file_name)

In [111]:
# LOCAL_FILENAME_docker = f'{REPO_DOCKER_PATH_PREFIX}/Dockerfile.{SERVING_DOCKERNAME}'
# LOCAL_FILENAME_main = f'{REPO_DOCKER_PATH_PREFIX}/{SERVING_SUB_DIR}/main.py'
# LOCAL_FILENAME_predictor = f'{REPO_DOCKER_PATH_PREFIX}/{SERVING_SUB_DIR}/predictor.py'
# LOCAL_FILENAME_prestart = f'{REPO_DOCKER_PATH_PREFIX}/{SERVING_SUB_DIR}/prestart.sh'
# LOCAL_FILENAME_reqs = f'{REPO_DOCKER_PATH_PREFIX}/{SERVING_SUB_DIR}/requirements.txt'

# GCS_MODEL_DIR_PATH = 'test-2tower-merlin-tf-jtv1/run-v5-20221108-163948/model-dir'
# GCS_SERVING_DIR_PATH = f'{GCS_MODEL_DIR_PATH}/serving'

# DESTINATION_FILENAME_docker = f'{GCS_SERVING_DIR_PATH}/Dockerfile.{SERVING_DOCKERNAME}'
# DESTINATION_FILENAME_main = f'{GCS_SERVING_DIR_PATH}/app/main.py'
# DESTINATION_FILENAME_predictor = f'{GCS_SERVING_DIR_PATH}/app/predictor.py'
# DESTINATION_FILENAME_prestart = f'{GCS_SERVING_DIR_PATH}/app/prestart.sh'
# DESTINATION_FILENAME_reqs = f'{GCS_SERVING_DIR_PATH}/app/requirements.txt'

# # print(f"LOCAL_FILENAME_docker : {LOCAL_FILENAME_docker}")
# # print(f"LOCAL_FILENAME_main : {LOCAL_FILENAME_main}")
# # print(f"LOCAL_FILENAME_predictor : {LOCAL_FILENAME_predictor}")
# # print(f"LOCAL_FILENAME_prestart : {LOCAL_FILENAME_prestart}")
# # print(f"LOCAL_FILENAME_reqs : {LOCAL_FILENAME_reqs}\n")
# print(f"GCS_MODEL_DIR_PATH : {GCS_MODEL_DIR_PATH}\n")
# print(f"DESTINATION_FILENAME_docker : {DESTINATION_FILENAME_docker}")

In [112]:
# _upload_blob_gcs(BUCKET_URI, LOCAL_FILENAME_docker,DESTINATION_FILENAME_docker, PROJECT_ID)
# _upload_blob_gcs(BUCKET_URI, LOCAL_FILENAME_main,DESTINATION_FILENAME_main, PROJECT_ID)
# _upload_blob_gcs(BUCKET_URI, LOCAL_FILENAME_predictor,DESTINATION_FILENAME_predictor, PROJECT_ID)
# _upload_blob_gcs(BUCKET_URI, LOCAL_FILENAME_prestart,DESTINATION_FILENAME_prestart, PROJECT_ID)
# _upload_blob_gcs(BUCKET_URI, LOCAL_FILENAME_reqs,DESTINATION_FILENAME_reqs, PROJECT_ID)

In [113]:
# !gsutil ls gs://jt-merlin-scaling/test-2tower-merlin-tf-jtv1/run-v5-20221108-163948/model-dir/serving

### Build Serving Image

In [114]:
# Docker definitions for training
SERVING_VERSION = 'v6'
IMAGE_NAME = f'merlin-triton-serving-{SERVING_VERSION}'
IMAGE_URI = f'gcr.io/{PROJECT_ID}/{IMAGE_NAME}'

DOCKERNAME = f'{SERVING_DOCKERNAME}'
MACHINE_TYPE ='e2-highcpu-32'
FILE_LOCATION = './src'

In [115]:
# FILE_LOCATION = './src'
# ! gcloud builds submit --config src/cloudbuild.yaml --substitutions _DOCKERNAME=$DOCKERNAME,_IMAGE_URI=$IMAGE_URI,_FILE_LOCATION=$FILE_LOCATION --timeout=2h --machine-type=e2-highcpu-8

In [116]:
! gcloud builds submit --config src/cloudbuild.yaml \
    --substitutions _DOCKERNAME=$DOCKERNAME,_IMAGE_URI=$IMAGE_URI,_FILE_LOCATION=$FILE_LOCATION \
    --timeout=2h \
    --machine-type=$MACHINE_TYPE

Creating temporary tarball archive of 40 file(s) totalling 1.1 MiB before compression.
Uploading tarball of [.] to [gs://hybrid-vertex_cloudbuild/source/1668026173.08763-f1cbda98a6584b2d9b65b0886c42caab.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/hybrid-vertex/locations/global/builds/2681eb09-22d5-4727-8e70-503dcc2365e6].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/2681eb09-22d5-4727-8e70-503dcc2365e6?project=934903580331 ].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "2681eb09-22d5-4727-8e70-503dcc2365e6"

FETCHSOURCE
Fetching storage object: gs://hybrid-vertex_cloudbuild/source/1668026173.08763-f1cbda98a6584b2d9b65b0886c42caab.tgz#1668026173582399
Copying gs://hybrid-vertex_cloudbuild/source/1668026173.08763-f1cbda98a6584b2d9b65b0886c42caab.tgz#1668026173582399...
/ [1 files][198.7 KiB/198.7 KiB]                                                
Operation completed over 1 objects/198.7 K

# Deploy Query Model with Vertex AI

## Upload to Model Regsitry

In [117]:
MODEL_DISPLAY_NAME = "mm_qtower_test_v7"
MODEL_ARTIFACT_URI = 'gs://jt-merlin-scaling/test-2tower-merlin-tf-jtv1/run-v5-20221108-163948/model-dir'

In [118]:
model = vertex_ai.Model.upload(
        display_name=MODEL_DISPLAY_NAME,
        artifact_uri=MODEL_ARTIFACT_URI,
        serving_container_image_uri=IMAGE_URI,
        serving_container_predict_route='/predict',
        serving_container_health_route='/health',
        serving_container_command=["sh", "-c", "uvicorn app.main:app --host 0.0.0.0 --port $AIP_HTTP_PORT"],
        serving_container_args='--gpus all',
        sync=True,
    )

Creating Model
Create Model backing LRO: projects/934903580331/locations/us-central1/models/4060694353469767680/operations/3121281857899986944
Model created. Resource name: projects/934903580331/locations/us-central1/models/4060694353469767680@1
To use this Model in another session:
model = aiplatform.Model('projects/934903580331/locations/us-central1/models/4060694353469767680@1')


## Deploy to Vertex AI Endpoint

In [119]:
endpoint = model.deploy(
    deployed_model_display_name="mm_query_tower_deploy",
    machine_type="n1-standard-4",
    min_replica_count=1,
    max_replica_count=1,
    accelerator_type="NVIDIA_TESLA_T4",
    accelerator_count=1,
)

Creating Endpoint
Create Endpoint backing LRO: projects/934903580331/locations/us-central1/endpoints/6328054455711301632/operations/6622830568180547584
Endpoint created. Resource name: projects/934903580331/locations/us-central1/endpoints/6328054455711301632
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/934903580331/locations/us-central1/endpoints/6328054455711301632')
Deploying model to Endpoint : projects/934903580331/locations/us-central1/endpoints/6328054455711301632
Deploy Endpoint model backing LRO: projects/934903580331/locations/us-central1/endpoints/6328054455711301632/operations/4678682904040046592


FailedPrecondition: 400 Model server terminated: model server container terminated: exit_code: 	 1
reason: "Error"
started_at {
  seconds: 1668029551
}
finished_at {
  seconds: 1668029695
}
. Model server logs can be found at https://console.cloud.google.com/logs/viewer?project=934903580331&resource=aiplatform.googleapis.com%252FEndpoint&advancedFilter=resource.type%3D%22aiplatform.googleapis.com%2FEndpoint%22%0Aresource.labels.endpoint_id%3D%226328054455711301632%22%0Aresource.labels.location%3D%22us-central1%22.

## Test Deployed Endpoint

In [ ]:
## Ground truth candidate:
    # 'album_uri_can': 'spotify:album:5l83t3mbVgCrIe1VU9uJZR', 
    # 'artist_name_can': 'Russ', 
    # 'track_name_can': 'We Just Havent Met Yet', 
## TODO - we have to overload with candidate data because of the workflow transform, add overloaded values in the predictor
TEST_INSTANCE = {'collaborative': 'false',
                 'album_name_pl': ["There's Really A Wolf", 'Late Nights: The Album',
                       'American Teen', 'Crazy In Love', 'Pony'], 
                 'artist_genres_pl': ["'hawaiian hip hop', 'rap'",
                       "'chicago rap', 'dance pop', 'pop', 'pop rap', 'r&b', 'southern hip hop', 'trap', 'urban contemporary'",
                       "'pop', 'pop r&b'", "'dance pop', 'pop', 'r&b'",
                       "'chill r&b', 'pop', 'pop r&b', 'r&b', 'urban contemporary'"], 
                 'artist_name_pl': ['Russ', 'Jeremih', 'Khalid', 'Beyonc\xc3\xa9',
                       'William Singe'], 
                 'artist_pop_can': 82.0, 
                 'description_pl': '', 
                 'duration_ms_songs_pl': [237506.0, 217200.0, 219080.0, 226400.0, 121739.0], 
                 'n_songs_pl': 8.0, 
                 'name': 'Lit Tunes ', 
                 'num_albums_pl': 8.0, 
                 'num_artists_pl': 8.0, 
                 'track_name_pl': ['Losin Control', 'Paradise', 'Location',
                       'Crazy In Love - Remix', 'Pony'], 
                 'track_pop_pl': [79.0, 58.0, 83.0, 71.0, 57.0],
                 'duration_ms_seed_pl': 51023.1,
                 'pid': 1,
                 'track_uri_pl': ['spotify:track:4cxMGhkinTocPSVVKWIw0d',
                       'spotify:track:1wNEBPo3nsbGCZRryI832I',
                       'spotify:track:152lZdxL1OR0ZMW6KquMif',
                       'spotify:track:2f4IuijXLxYOeBncS60GUD',
                       'spotify:track:4Lj8paMFwyKTGfILLELVxt']
                     }

In [ ]:
#make a prediction

endpoint.predict(instances=[[TEST_INSTANCE, TEST_INSTANCE]])